# 6강 : "물류를 배송하기 위한 최적의 경로를 알아낼 수 있을까요?" 

# 1. 데이터 읽기

In [122]:
# 물류데이터 4개 

# tbl_factory.csv : 생산 공장 데이터 
# tbl_warehouse.csv : 창고 데이터
# rel_cost.csv : 창고와 공장 간의 운송비용 
# tbl_trasaction.csv : 2019년의 공장으로의 부품 운송 실적  --> 메인 데이터 

In [123]:
import pandas as pd 

# 1. 공장 데이터
factories = pd.read_csv("tbl_factory.csv") 
factories.head()

,FCID,FCName,FCDemand,FCRegion
0,FC00001,서울공장,28,북부
1,FC00002,수원공장,29,북부
2,FC00003,인천공장,31,북부
3,FC00004,대전공장,25,북부
4,FC00005,대구공장,21,남부


In [124]:
# 단, 데이터 고유키를 인덱스로 설정하여, 행을 식별할수 있게끔 저장 
factories = pd.read_csv("tbl_factory.csv", index_col = 0) 
factories.head()

,FCName,FCDemand,FCRegion
FCID,,,
FC00001,서울공장,28,북부
FC00002,수원공장,29,북부
FC00003,인천공장,31,북부
FC00004,대전공장,25,북부
FC00005,대구공장,21,남부


In [125]:
# 2. 창고 데이터
warehouses = pd.read_csv("tbl_warehouse.csv", index_col = 0) 
warehouses.head()

,WHName,WHSupply,WHRegion
WHID,,,
WH00001,판교창고,35,북부
WH00002,용인창고,41,북부
WH00003,수원창고,42,북부
WH00004,대전창고,60,남부
WH00005,대구창고,72,남부


In [126]:
# 3. 비용 데이터 (창고와 공장 간의 운송비용)
cost = pd.read_csv("rel_cost.csv", index_col = 0) 
cost.head()

# FC00001 공장 --> WH00001 창고로 이동시 비용 0.4 

,FCID,WHID,Cost
RCostID,,,
1,FC00001,WH00001,0.4
2,FC00001,WH00002,0.8
3,FC00001,WH00003,1.5
4,FC00002,WH00001,0.9
5,FC00002,WH00002,0.8


In [127]:
# 4. 운송실적 (2019년 1월 1일 ~ 2019년 12월 31일) 

trans = pd.read_csv("tbl_transaction.csv", index_col = 0)
trans.head()

# ToFC 출발 공장 
# FromWH 도착 창고  
# Quantity 운송 수 

,TransactionDate,ToFC,FromWH,Quantity
TRID,,,,
0,2019-01-01 02:11:10,FC00004,WH00003,33
1,2019-01-01 06:12:42,FC00007,WH00006,19
2,2019-01-01 06:32:32,FC00006,WH00004,31
3,2019-01-01 07:17:06,FC00002,WH00003,18
4,2019-01-01 07:52:18,FC00001,WH00002,30


# 2. 데이터 가공

In [128]:
# 메인데이터인 운송실적 데이터를 기준으로, 비용 데이터를 붙힙니다. 

join_data = pd.merge(trans, cost , left_on = ["ToFC", "FromWH"], right_on = ["FCID", "WHID"], how="left")
#             --- 병합 --왼쪽 -- 오른쪽            ----------왼쪽데이터 열이름       -------오른쪽데이터  
#                                                 --------------------------------- 컬럼명은 달라도 값 일치하면 서로 붙음 
join_data

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8
...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,FC00007,WH00006,29,FC00007,WH00006,1.3
3996,2019-12-31 17:10:21,FC00002,WH00003,35,FC00002,WH00003,1.6
3997,2019-12-31 19:35:07,FC00003,WH00001,36,FC00003,WH00001,1.2
3998,2019-12-31 21:45:07,FC00005,WH00005,34,FC00005,WH00005,0.3


In [129]:
# 그리고 공장데이터도 추가합니다. 

join_data = pd.merge(join_data, factories, left_on = "ToFC", right_on = "FCID", how="left")
join_data.head()

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost,FCName,FCDemand,FCRegion
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1,대전공장,25,북부
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3,부산공장,25,남부
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9,광주공장,30,남부
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6,수원공장,29,북부
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8,서울공장,28,북부


In [130]:
# 마지막으로 창고 데이터도 추가합니다. 

join_data = pd.merge(join_data, warehouses, left_on = "FromWH", right_on = "WHID" , how="left")
join_data.head()

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost,FCName,FCDemand,FCRegion,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1,대전공장,25,북부,수원창고,42,북부
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3,부산공장,25,남부,광주창고,65,남부
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9,광주공장,30,남부,대전창고,60,남부
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6,수원공장,29,북부,수원창고,42,북부
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8,서울공장,28,북부,용인창고,41,북부


In [131]:
# 필요한 컬럼만 배치합니다. 

join_data = join_data [[
    "TransactionDate", "Quantity", "Cost", "ToFC", "FCName", "FCDemand", "FromWH", "WHName", "WHSupply", "WHRegion"
]]
join_data.head()

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부


In [132]:
# 가독성 좋게 한글로 바꿉니다. 

join_data = join_data.rename(columns = {
    "TransactionDate" : "운송날짜", 
    "Quantity" : "운송건수", 
    "Cost" : "운송비용", 
    "ToFC": "공장ID", 
    "FCName" : "공장명", 
    "FCDemand" : "공장수요", 
    "FromWH" : "창고ID", 
    "WHName" : "창고명", 
    "WHSupply" : "창고 공급량", 
    "WHRegion" : "지사"
})

join_data.head()

,운송날짜,운송건수,운송비용,공장ID,공장명,공장수요,창고ID,창고명,창고 공급량,지사
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부


In [133]:
# 몇 개의 공장이 있는지 궁급합니다. 

print(join_data["공장ID"].unique())
print(join_data["공장명"].unique())
print(join_data["공장명"].count())

['FC00004' 'FC00007' 'FC00006' 'FC00002' 'FC00001' 'FC00005' 'FC00003'
 'FC00008']
['대전공장' '부산공장' '광주공장' '수원공장' '서울공장' '대구공장' '인천공장' '울산공장']
4000


In [134]:
# 몇 개의 창고가 있는지 궁급합니다. 

print(join_data["창고ID"].unique())
print(join_data["창고명"].unique())
print(join_data["창고명"].count())

['WH00003' 'WH00006' 'WH00004' 'WH00002' 'WH00005' 'WH00001']
['수원창고' '광주창고' '대전창고' '용인창고' '대구창고' '판교창고']
4000


In [135]:
# 몇 개의 지사가 있는지 궁급합니다. 

print(join_data["지사"].unique())
print(join_data["지사"].count())
print((join_data["지사"]=="북부").sum())
print((join_data["지사"]=="남부").sum())

['북부' '남부']
4000
2000
2000


In [136]:
# 북부지사와 남부지사별로 데이터 나눠서 저장해둡니다. 

# north = join_data["지사"]=="북부"  # --> 이렇게 하면 "북부"가 아닌 "남부" 값은 False로 나옴 
# [ ] 을 한 번 더 씌운다. 
north = join_data[join_data["지사"]=="북부"] 

print(len(north))
print(north["지사"].unique())

north.head()

2000
['북부']


,운송날짜,운송건수,운송비용,공장ID,공장명,공장수요,창고ID,창고명,창고 공급량,지사
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부
7,2019-01-01 09:09:30,12,1.5,FC00001,서울공장,28,WH00003,수원창고,42,북부
8,2019-01-01 10:52:55,27,1.5,FC00003,인천공장,31,WH00003,수원창고,42,북부


In [137]:
south = join_data[join_data["지사"]=="남부"] 

print(len(south))
print(south["지사"].unique())

south.head()

2000
['남부']


,운송날짜,운송건수,운송비용,공장ID,공장명,공장수요,창고ID,창고명,창고 공급량,지사
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
5,2019-01-01 08:56:09,31,0.3,FC00005,대구공장,21,WH00005,대구창고,72,남부
6,2019-01-01 09:00:15,33,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부
9,2019-01-01 14:12:51,21,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부


# 3. 데이터 집계

In [138]:
# 북부지사와 남부지사의 1년간 운송 총 건수

print("북부지사 운송건수 : " + str(north["운송건수"].sum()) + "건" )
print("남부지사 운송건수 : " + str(south["운송건수"].sum()) + "건" )

# 남부지사가 더 많이 운송하였습니다.

북부지사 운송건수 : 49146건
남부지사 운송건수 : 50214건


In [139]:
# 북부지사와 남부지사의 1년간 총 운송 비용 (단위: 1 = 10,000원) 

print("북부지사 운송비용 : " + str(north["운송비용"].sum()) + "만원" )
print("남부지사 운송비용 : " + str(south["운송비용"].sum()) + "만원" )

# 하지만 운송하는데 소요되는 비용은 북부지사가 더 많습니다.

북부지사 운송비용 : 2189.3만원
남부지사 운송비용 : 2062.0만원


In [140]:
# 운송 1건당 발생비용 = 총 비용 / 총 건수 

north_1_cost = (north["운송비용"].sum() / north["운송건수"].sum()) * 10000 
round(north_1_cost,1)    # 445.5 
print("북부지사 운송 1건당 비용",round(north_1_cost,1),"만원")

south_1_cost = (south["운송비용"].sum() / south["운송건수"].sum()) * 10000 
round(south_1_cost,1)    # 445.5 
print("남부지사 운송 1건당 비용",round(south_1_cost,1),"만원")

북부지사 운송 1건당 비용 445.5 만원
남부지사 운송 1건당 비용 410.6 만원


In [141]:
# 두 지사의 평균 운송비용 확인하기

join_data.head()

,운송날짜,운송건수,운송비용,공장ID,공장명,공장수요,창고ID,창고명,창고 공급량,지사
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부


In [142]:
# 지사가 "북부"인 데이터에 한하여 "운송비용" 
join_data["운송비용"].loc[join_data["지사"]== "북부"]

0       1.1
3       1.6
4       0.8
7       1.5
8       1.5
       ... 
3984    0.9
3987    1.3
3992    0.8
3996    1.6
3997    1.2
Name: 운송비용, Length: 2000, dtype: float64

In [143]:
# 지사가 "북부"인 데이터에 한하여 "평균 운송비용" 

join_data["운송비용"].loc[join_data["지사"]=="북부"].mean()   # 단위 1원 

1.0946500000000001

In [145]:
join_data["운송비용"].loc[join_data["지사"]=="남부"].mean()   # 단위 1원 

# 남부가 평균 운송비용 또한 낮아, 효율적인 운영을 하고 있습니다.

1.031

# 4. 물류 최적 경로

## 4.1 네트워크 가시화 NetworkX